# **Reinforcement Learning in Collectible Card Games**
Below is the code for some of the AI agents described in the project paper. You will be able to find the code for all the agents however some of the previous iterations of these agents are omitted. You can play around yourself with the hyper-parameters and weights but note that the values given in this code were the final ones used in the paper.

**Personal note:** massive shoutout to Vieira, Ronaldo and Chaimowicz, Luiz and Tavares, Anderson Rocha - the author's of gym-locm "OpenAI Gym Environments for Legends of Code and Magic". Both their code and reimplementation of the LOCM enviornment was vital for the success of this project. You can find it here https://github.com/ronaldosvieira/gym-locm .

*It is important to open drive and install gym-locm so that the AI can play games*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My Drive/locm_project
! git clone https://github.com/ronaldosvieira/gym-locm.git
%cd gym-locm
! pip install -e .

/content/drive/My Drive/locm_project
fatal: destination path 'gym-locm' already exists and is not an empty directory.
/content/drive/My Drive/locm_project/gym-locm
Obtaining file:///content/drive/My%20Drive/locm_project/gym-locm
  Running setup.py develop for gym-locm


*Imports*

In [ ]:
import numpy as np
import collections
from collections import defaultdict
import random
import math
import time
from operator import attrgetter
from typing import Type

import pexpect
import sys

from gym_locm.engine import *

import gym
import gym.spaces

import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt

**Random Battle Agent** - plays cards and picks actions randomly.

In [ ]:
class RandomBattleAgent():
    def __init__(self, seed=None):
        self.random = random.Random(seed)

    def seed(self, seed):
        self.random.seed(seed)

    def reset(self):
        pass

    def act(self, state):
        index = int(len(state.available_actions) * random.random())

        return state.available_actions[index]

**Greedy Battle Agent** - plays cards that maximize the value of a handcrafted heuristic (see paper for more details).

In [ ]:
class GreedyBattleAgent():
    def seed(self, seed):
        pass

    def reset(self):
        pass

    @staticmethod
    def eval_state(state):
        
        def eval_creature(creature):
            score = 0

            if creature.attack > 0:
                score += 28.6875
                score += creature.attack * 23.375
                score += creature.defense * 29.625

                if creature.has_ability('W'):
                    score += creature.attack * 26

                if creature.has_ability('L'):
                    score += 29.0625

            if creature.has_ability('G'):
                score += 25.125

            return score
        
        score = 0

        pl = state.current_player
        op = state.opposing_player

        if pl.health < 5:
            score -= 100

        # check opponent's death
        if op.health <= 0:
            score += 100000

        # check own death
        elif pl.health <= 0:
            score -= 100000

        # health difference
        score += (pl.health - op.health) * 2
        
        for c in pl.hand:
            if not isinstance(c, Creature):
                score += 14.6875

        if len(pl.hand) + pl.bonus_draw + 1 <= 8:
            score += (pl.bonus_draw + 1) * 20.625
                
        for pl_lane, op_lane in zip(pl.lanes, op.lanes):
            # creature strength
            score += sum(eval_creature(c) for c in pl_lane)
            score -= sum(eval_creature(c) for c in op_lane)

        return score

    def act(self, state):
        best_action, best_score = None, float("-inf")

        if len(state.available_actions) == 1:
            return state.available_actions[0]
        
        for action in state.available_actions:
            state_copy = state.clone()
            state_copy.act(action)

            score = self.eval_state(state_copy)
            if score > best_score:
                best_action, best_score = action, score
        return best_action

*Datastructure required for hashing of game states*

In [ ]:
class Node:
    def __init__(self, state, actions, parent):
        self.state = state
        self.actions = actions
        self.parent = parent
        self.player_id = state.opposing_player.id if actions[:-1].count(Action(ActionType.PASS)) % 2 else state.current_player.id 

        self._hash = None
        
    def __hash__(self):
        """Nodes must be hashable"""
        if self._hash is not None:
            return self._hash

        s = self.state
        p0, p1 = self.state.players
        cp = self.state.players[self.player_id]

        attributes = [
            s.phase, s.turn, s.current_player.id,
            p0.health, p0.base_mana + p0.bonus_mana, p0.bonus_draw,
            p1.health, p1.base_mana + p1.bonus_mana, p1.bonus_draw
        ]

        attributes.extend(c.instance_id
                          for c in sorted(cp.hand, key=attrgetter('id')))

        for p in (p0, p1):
            for j in range(2):
                for i in range(3):
                    if len(p.lanes[j]) > i:
                        c = sorted(p.lanes[j], key=attrgetter('id'))[i]

                        stats = [c.instance_id, c.attack, c.defense] + \
                            list(map(int, map(c.keywords.__contains__, 'BCDGLW'))) + \
                            [int(p.id), j, c.able_to_attack()]
                    else:
                        stats = [-1] * 12

                    attributes.extend(stats)

        for action in self.actions:
            attributes.extend((action.type, action.origin, action.target))

        self._hash = hash(tuple(attributes))

        return self._hash

    def __eq__(self, other):
        """Nodes must be comparable"""
        return hash(self) == hash(other)

*Some additional helper functions*

In [ ]:
def eval_state(state, pl, op):
    def eval_creature(creature):
        score = 0

        if creature.attack > 0:
            score += 20
            score += creature.attack * 10
            score += creature.defense * 5

            if creature.has_ability('W'):
                score += creature.attack * 5

            if creature.has_ability('L'):
                score += 20

        if creature.has_ability('G'):
            score += 9

        return score
                
    score = 0

    if pl.health < 5:
        score -= 100

        # check opponent's death
    if op.health <= 0:
        score += 100000

        # check own death
    elif pl.health <= 0:
        score -= 100000

        # health difference
    score += (pl.health - op.health) * 2
                
    for c in pl.hand:
        if not isinstance(c, Creature):
            score += 21

    if len(pl.hand) + pl.bonus_draw + 1 <= 8:
        score += (pl.bonus_draw + 1) * 5
                        
    for pl_lane, op_lane in zip(pl.lanes, op.lanes):
        # creature strength
        score += sum(eval_creature(c) for c in pl_lane)
        score -= sum(eval_creature(c) for c in op_lane)

    return score

def eval_node(node):
    state_copy = node.state.clone()

    actions = node.actions[:-1] if node.actions[-1] == Action(ActionType.PASS) else node.actions

    for action in actions:
        state_copy.act(action)

    return eval_state(state_copy, state_copy.current_player, state_copy.opposing_player)

**Minimax tree search agent** - details of the play policy can be found in the paper.

In [ ]:
class TreeSearch:

    def __init__(self, player_id, simulations=10):
        self.agent = GreedyBattleAgent()
        self.Q = defaultdict(float)  # total reward of each node
        self.children = defaultdict(list)
        self.player_id = player_id
        self.simulations = simulations

    def choose(self, state):
        # choose an action in the real game
        node = Node(state, [], None)

        def score(n):
            return self.Q[n]

        return max(self.children[node], key=score).actions[-1]
            
    def search(self, state):
        node = Node(state, [], None)

        self._expand(node)
        
        for child in self.children[node]:
            total_reward = 0
            for _ in range(self.simulations):
                new_state = child.state.clone()
                total_reward += self._simulate(new_state)
            self.Q[child] = total_reward / self.simulations
        
    def _expand(self, node):
        # expand a node
        if node in self.children:
            return  # already expanded
        
        children = []

        for action in node.state.available_actions:
            new_state = node.state.clone()
            new_state.act(action)
            children.append(Node(new_state, node.actions + [action], node))

        self.children[node] = children

    def _simulate(self, new_state):
        # simulate opponent move

        amount_deck = len(new_state.opposing_player.deck)
        amount_hand = len(new_state.opposing_player.hand)

        ids = map(attrgetter('instance_id'),
                  new_state.opposing_player.hand +
                  new_state.opposing_player.deck)

        new_deck = []

        for i, id in zip(range(amount_deck + amount_hand), ids):
            random_index = int(3 * random.random())

            card = new_state._draft_cards[i][random_index].make_copy(id)

            new_deck.append(card)

        random.shuffle(new_deck)

        new_state.opposing_player.deck = new_deck
        new_state.opposing_player.hand = []

        new_state.opposing_player.draw(amount=amount_hand)

        for player in new_state.players:
            random.shuffle(player.deck)

        def eval_state(state, pl, op):
            def eval_creature(creature):
                score = 0

                if creature.attack > 0:
                    score += 20
                    score += creature.attack * 10
                    score += creature.defense * 5

                    if creature.has_ability('W'):
                        score += creature.attack * 5

                    if creature.has_ability('L'):
                        score += 20

                if creature.has_ability('G'):
                    score += 9

                return score
                
            score = 0

            if pl.health < 5:
                score -= 100

            # check opponent's death
            if op.health <= 0:
                score += 100000

            # check own death
            elif pl.health <= 0:
                score -= 100000

            # health difference
            score += (pl.health - op.health) * 2
                
            for c in pl.hand:
                if not isinstance(c, Creature):
                    score += 21

            if len(pl.hand) + pl.bonus_draw + 1 <= 8:
                score += (pl.bonus_draw + 1) * 5
                        
            for pl_lane, op_lane in zip(pl.lanes, op.lanes):
                # creature strength
                score += sum(eval_creature(c) for c in pl_lane)
                score -= sum(eval_creature(c) for c in op_lane)

            return score
        
        while new_state.current_player.id == self.player_id:
            action = self.agent.act(new_state)
            new_state.act(action)
            if new_state.winner is not None:
                return eval_state(new_state, new_state.current_player, new_state.opposing_player)

        while new_state.current_player.id != self.player_id:
            # opponent greedily makes action
            action = self.agent.act(new_state)
            new_state.act(action)
            if new_state.winner is not None:
                return eval_state(new_state, new_state.opposing_player, new_state.current_player)

        while new_state.current_player.id == self.player_id:
            # tree search greedily makes action
            action = self.agent.act(new_state)
            new_state.act(action)
            if new_state.winner is not None:
                return eval_state(new_state, new_state.current_player, new_state.opposing_player)

        while new_state.current_player.id != self.player_id:
            # opponent greedily makes action
            action = self.agent.act(new_state)
            new_state.act(action)
            if new_state.winner is not None:
                return eval_state(new_state, new_state.opposing_player, new_state.current_player)

        while new_state.current_player.id == self.player_id:
            # tree search greedily makes action
            action = self.agent.act(new_state)
            new_state.act(action)
            if new_state.winner is not None:
                return eval_state(new_state, new_state.current_player, new_state.opposing_player)
    
        return eval_state(new_state, new_state.opposing_player, new_state.current_player)

class TreeSearchBattleAgent():
    def __init__(self):
        pass

    def seed(self, seed):
        pass
    
    def reset(self):
        pass
    
    def act(self, state):
        searcher = TreeSearch(state.current_player.id)

        if len(state.available_actions) == 1:
            return state.available_actions[0]

        searcher.search(state)
        return searcher.choose(state)

**(best) Monte-Carlo Tree-Search agent** - standard MCTS algorithm that uses the following modifications (details of which can be found in the paper): move ordering, probabalistic early cutoff simulation, progressive UCT.



In [ ]:
class BestMCTS:

    def __init__(self, player_id, C=1.41, Lambda=0.01):
        self.N = defaultdict(int) # total visits of each node
        self.Q = defaultdict(int)  # total reward of each node
        self.children = defaultdict(list)
        self.H = defaultdict(float)
        self.player_id = player_id
        self.C = C
        self.Lambda = Lambda

    def choose(self, state):
        node = Node(state, [], None)

        if node not in self.children:
            index = int(len(state.available_actions) * random.random())

            return state.available_actions[index]

        def score(n):
            if self.N[n] == 0:
                return float("-inf")  # avoid unseen moves
            return self.Q[n] / self.N[n]  # average reward

        return max(self.children[node], key=score).actions[-1]

    def search(self, state):
        node = Node(state, [], None)

        path, new_state = self._select(node)
        leaf = path[-1]

        self._expand_with_ordering(leaf, state, new_state)
        reward = self._p_simulate(new_state, 0.1)
            
        self._backpropogate(path, reward)

    def _select(self, node):
        path = []

        state_copy = node.state.clone()

        while True:
            path.append(node)

            if node not in self.children or not self.children[node] or state_copy.winner is not None:
                return path, state_copy

            unexplored = [item for item in self.children[node] if item not in self.children.keys()]

            if unexplored:
                n = unexplored.pop()
                state_copy.act(n.actions[-1])
                path.append(n)
                return path, state_copy

            node = self._progressive_uct_select(node)          
            state_copy.act(node.actions[-1])
            
    def _expand_with_ordering(self, node, root_state, new_state):
        if node in self.children:
            return

        children = []

        for action in new_state.available_actions:
            if action.type == ActionType.SUMMON or action.type == ActionType.USE:
                children.append(Node(root_state, node.actions + [action], node))

        if children == []:
            for action in new_state.available_actions:
                children.append(Node(root_state, node.actions + [action], node))
            

        self.children[node] = children

    def _p_simulate(self, new_state, p):
        amount_deck = len(new_state.opposing_player.deck)
        amount_hand = len(new_state.opposing_player.hand)

        ids = map(attrgetter('instance_id'),
                  new_state.opposing_player.hand +
                  new_state.opposing_player.deck)

        new_deck = []

        for i, id in zip(range(amount_deck + amount_hand), ids):
            random_index = int(3 * random.random())

            card = new_state._draft_cards[i][random_index].make_copy(id)

            new_deck.append(card)

        random.shuffle(new_deck)

        new_state.opposing_player.deck = new_deck
        new_state.opposing_player.hand = []

        new_state.opposing_player.draw(amount=amount_hand)

        for player in new_state.players:
            random.shuffle(player.deck)

        agent = RandomBattleAgent()
        
        while True:
            if new_state.winner is not None:
                return 1 if new_state.winner == self.player_id else -1
            action = agent.act(new_state)
            new_state.act(action)
            if action.type == ActionType.PASS and random.random() < p:
                break

        while new_state.current_player.id == self.player_id:
            if new_state.winner is not None:
                return 1
            action = agent.act(new_state)
            new_state.act(action)
            
        return 1 if eval_state(new_state, new_state.opposing_player, new_state.current_player) > eval_state(new_state, new_state.current_player, new_state.opposing_player) else -1
        
    def _backpropogate(self, path, reward):
        for node in reversed(path):
            self.N[node] += 1

            if node.player_id == self.player_id:
                self.Q[node] += reward
            else:
                self.Q[node] -= reward

    def _progressive_uct_select(self, node):
        log_n_vertex = math.log(self.N[node])
        def uct(n):
            if n not in self.H:
                self.H[n] = eval_node(n)
            return (self.Q[n] / self.N[n]) + self.C * math.sqrt(log_n_vertex / self.N[n]) + self.Lambda * (self.H[n] / (self.N[n] + 1))

        return max(self.children[node], key=uct)
        
class BestMCTSBattleAgent():
    def __init__(self):
        self.count = 0
        self.total = 0

    def seed(self, seed):
        pass

    def reset(self):
        pass

    def avg_simulations_per_second(self):
        return self.count / self.total

    def act(self, state, time_limit_ms=1000):
        searcher = BestMCTS(state.current_player.id)
        
        if len(state.available_actions) == 1:
            return state.available_actions[0]
        
        start_time = int(time.time() * 1000.0)
        while True:
            current_time = int(time.time() * 1000.0)

            if current_time - start_time > time_limit_ms:
                break
            
            searcher.search(state)
            self.count += 1
        self.total += 1
        return searcher.choose(state)

*some additional helper functions for the deep RL agents*

In [ ]:
name = "LOCM-battle-v0"
env = gym.make(name)

n_actions = env.action_space.n
observation_space_shape = np.array(env.observation_space.shape).prod()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def available_actions():
  available_actions = env.state.available_actions
  action_numbers = []

  for action_number in range(n_actions):
    action = env.decode_action(action_number)
    if action in available_actions:
      action_numbers.append(action_number)

  return np.array(action_numbers, dtype=np.int64)

**Q Network agent** - deep RL agent that uses a neural network for the play policy (takes argmax of network output).

In [ ]:
class QNetwork(nn.Module):
  def __init__(self):
    super(QNetwork, self).__init__()

    self.fc = nn.Sequential(
        nn.Linear(observation_space_shape, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, n_actions)
    )
  
  def forward(self, x, mask):
    x = x.view(x.size(0),-1)
    fx = x.float() / 256
    q_vals_v = self.fc(fx)

    masked_q_vals_v = q_vals_v.masked_fill(~mask, -np.inf)
    return masked_q_vals_v

class QNetworkBattleAgent():
  def __init__(self, path):
    self.net = QNetwork().to(device)
    params = torch.load(path, map_location=torch.device(device))
    self.net.load_state_dict(params['net'])
    print('[Q Network]: Loaded trained model network from '+path)

  def seed(self, seed):
    pass

  def reset(self):
    pass

  def act(self, state):
    state = env.encode_state()

    indices = torch.tensor(available_actions()).to(device)
    mask = torch.zeros(n_actions, dtype=torch.bool).to(device)
    mask.scatter_(0, indices, True)

    state_a = np.array([state], copy=False)
    state_v = torch.tensor(state_a).to(device)

    q_vals_v = self.net(state_v, mask)
    act_v = q_vals_v.argmax()
    action = int(act_v.item())

    return action

**Policy network agent** - deep RL agent that uses a neural network for the play policy (samples actions to play from a discrete distribution constructed by softmaxing network outputs).

In [ ]:
class PolicyNet(nn.Module):
  def __init__(self):
    super(PolicyNet, self).__init__()

    self.actor = nn.Sequential(
      nn.Linear(observation_space_shape, 512),
      nn.ReLU(),
      nn.Linear(512, 256),
      nn.ReLU(),
      nn.Linear(256, n_actions)
    )
    self.critic = nn.Sequential(
      nn.Linear(observation_space_shape, 512),
      nn.ReLU(),
      nn.Linear(512, 256),
      nn.ReLU(),
      nn.Linear(256, 1)
    )

  def forward(self, x, mask):
    x = x.view(x.size(0),-1)
    fx = x.float() / 256

    policy = self.actor(fx)
    value = self.critic(fx)

    masked_policy = policy.masked_fill(~mask, -np.inf)
    return masked_policy, value

class PolicyNetBattleAgent():
  def __init__(self, path):
    self.net = PolicyNet().to(device)
    params = torch.load(path, map_location=torch.device(device))
    self.net.load_state_dict(params['net'])
    print('[POLICY NET]: Loaded trained network from '+path)

  def seed(self, seed):
    pass

  def reset(self):
    pass

  def act(self, state):
    state = env.encode_state()

    indices = torch.tensor(available_actions()).to(device)
    mask = torch.zeros(n_actions dtype=torch.bool).to(device)
    mask.scatter_(0, indices, True)

    state_a = np.array([state], copy=False)
    state_v = torch.tensor(state_a).to(device)

    logits, _ = self.net(state_v, mask)
    probs = F.softmax(logits, dim=1)
    dist = Categorical(probs)
    action = int(dist.sample())

    return action

*Example code for playing games between AI agents*

In [ ]:
name = "LOCM-battle-v0"
env = gym.make(name, battle_agent=RandomBattleAgent())

agent = # let this equal any of the agents above
'''
Examples:
agent = GreedyBattleAgent()
agent = QNetworkBattleAgent('../../../../drive/My Drive/...')
'''

num_episodes = 100
total_wins = 0

start_time = time.time()

for episode in range(num_episodes):
    state = env.reset()
    while True:
        action = agent.act(state)
        new_state, reward, done, info = env.step(action)
        state = new_state
        if done:
            if reward == 1:
                total_wins += 1
            break
env.close()

print("Validation 1")
print("____ Agent vs ____ Agent", str(total_wins / num_episodes * 100)+'% win rate')
print("Execution Time", (time.time() - start_time))

Loaded trained model from ../../../../drive/My Drive/training/locm-dqn-save-self3.chkpt
Validation 1
DQN Agent vs Random Agent 93.75% win rate
Execution Time 413.63376688957214
